# Tellurium

In [1]:
from interfacemaster.cellcalc import get_primitive_hkl, get_normal_from_MI, MID
from interfacemaster.interface_generator import core, convert_vector_index, get_disorientation
from numpy import array, dot, round, var, average
from numpy.linalg import inv,norm

## Define an interface core class

Input two cif files.For GBs, input the same file; for interfaces, input two different files.


The first structure is non-changed while the second one can be deformed to form an approximate CSL

Please input files of CONVENTIONAL cells if you are not familiar with the indices in the primitive cell.

In [2]:
my_interface = core('cif_files/Te_mp-19_conventional_standard.cif',\
                    'cif_files/Te_mp-19_conventional_standard.cif')

Warning!, this programme will rewrite the POSCAR file in this dir!


/Users/jasonxie/.local/lib/python3.8/site-packages/pymatgen/io/cif.py:1121: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))


In [3]:
#my_interface.atoms_2 = [1,1,1] - my_interface.atoms_2

In [4]:
#get the rotation mtx
R = get_disorientation(L1 = my_interface.conv_lattice_1, L2 = my_interface.conv_lattice_2, 
                       v1 = [1,-1,0], hkl1 = [1, 1, -2], \
                       v2 = [-1,-2,0], hkl2 = [-2, 1, -2])

In [5]:
my_interface.parse_limit(du = 1e-2, S  = 1e-2, sgm1=200, sgm2=200, dd = 1e-2)

#Do searching!
#We already know that there is a CSL by rotation 180 degrees
my_interface.search_fixed(R)

Congrates, we found an appx CSL!

U1 = 
[[ 1  1  3]
 [ 2  0 -1]
 [ 0  1 -2]]; sigma_1 = 11

U2 = 
[[-1 -1  4]
 [ 1 -1  3]
 [ 0  1  2]]; sigma_2 = 11

D = 
[[ 0.99853888 -0.         -0.00110625]
 [-0.          1.         -0.        ]
 [ 0.00192983 -0.          1.00146112]]



'U1', 'U2' are two sets of indices of the CSL in the two crystals' frames,
'D' is the deformation applied to the crystal 2 to form an approximate CSL

As you can see, to form a twinning structure we need to deform the structure provided!

## Convert the miller indices to be expressed in the primitive cell

Now we need to specify the orientation of our interface, which is by giving a miller indices in expressed in the primitive cell.

In [6]:
"""
The interface plane is the [1,-1,2] plane in the conventional cell and now we \
transform it into primitive cell
"""
hkl = get_primitive_hkl(array([1, 1, -2]), my_interface.conv_lattice_1, my_interface.lattice_1)
hkl

array([-2,  1,  2])

## Compute the indices of the two slabs to make bicrystal

Now let's compute the indices of the two slabs forming the interface
If you want a near orthogonal bicrystal, please set 'orthogonal to yes, 
and adjust the lim & tol.

In [7]:
my_interface.compute_bicrystal(hkl, normal_ortho= True, lim = 50, tol_ortho = 1e-3)

cell 1:
[[ 7  1  1]
 [ 0  2  0]
 [-4  0  1]]
cell 2:
[[ 7 -1 -1]
 [ 7  1 -1]
 [ 4  0  1]]


Here we obtained the two sets of indices to build a bicrystal

## Make a bicrystal!

You can adjust the bicrystal by
1. do expansion by set 'xyz' = [dimX, dimY, dimZ]
2. do RBT by 'dzdy' = a translation vector 
(please confine dzdy in the interface plane, 
for RBT in the normal, please use 'dp1' and 'dp2', to make vacuum in the interface use 'dx');
3. introduce vacuum in the interface by 'dx' = the distance spacing the two crystals
4. adjust the termination by 'dp1', 'dp2' = length of shift of the position of 
the termination from the interface into the bulk
5. introduce vacuum to make a surface in the end of the bicrystal by
'vx' = length of the vaccumn

default: xyz = [1,1,1] and others = 0

In [8]:
my_interface.get_bicrystal(xyz_1 = [1,1,1], xyz_2 = [1,1,1], dx = 0)

Good! You have made a 'POSCAR' file of this bicrystal, we also generated two files named 'cell_1.cif' and 'cell_2.cif' providing the structure of the two slabs.

In [26]:
import shutil
try:
    shutil.rmtree('CNID_inputs')
except:
    print('No existing files')
#To sample the CNID, try
my_interface.sample_CNID(grid = [8,8], xyz_1 = [1,1,1], xyz_2 = [1,1,1], dx = 0.5, filetype = 'LAMMPS', filename = 'atoms')

CNID
[[ 1. -1.]
 [ 0. -2.]
 [ 1.  0.]]
making 64 files...
completed


Please check the generated 'POSCAR.x.y' files. 
The CNID here is expressed in the primitive cell's frame.
You can refer to the cartesian coordinates or in the conventional cell's frame

In [9]:
CNID_cartesian = round(my_interface.CNID,8)
CNID_cartesian

array([[-2.25618721,  6.76856163],
       [-3.90783088, -3.90783088],
       [-5.959899  , -0.        ]])

In [10]:
CNID_conv = round(dot(inv(my_interface.conv_lattice_1)\
                              , my_interface.CNID),8)
CNID_conv

array([[-1.,  1.],
       [-1., -1.],
       [-1., -0.]])